# Contextual Compression Retriever

Purpose: demonstrate `ContextualCompressionRetriever` which compresses retrieved documents with an LLM-based compressor before returning them. This reduces context size while keeping task-relevant information.

Use when:
- You need shorter, focused context for downstream generation,  
- You want to remove irrelevant sections from long documents while preserving key facts.

In [ ]:
!pip install langchain chromadb openai tiktoken pypdf langchain_google_genai langchain-community wikipedia

## Prerequisites & install

- Install packages (first cell).  
- Provide the LLM API key (e.g., `GEMINI_API_KEY`) for compressor LLM.  
- `faiss-cpu` recommended for local index building.

In [100]:
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
!pip install faiss-cpu

## Imports & components

This cell imports the FAISS adapter, embeddings, document types, `ContextualCompressionRetriever`, and an LLM-based extractor (`LLMChainExtractor`) used as a compressor.

In [112]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_google_genai import ChatGoogleGenerativeAI

## Step 1 — Prepare documents

Provide documents with metadata. The compressor will remove or condense irrelevant parts when producing compressed results.

In [113]:
from langchain.schema import Document

# Step 1: Your source documents with metadata
documents = [
    Document(
        page_content="Football is the most popular sport in the world, played by over 250 million players across more than 200 countries.",
        metadata={"topic": "General", "source": "FIFA Stats", "year": 2023}
    ),
    Document(
        page_content="Lionel Messi is known for his incredible dribbling, vision, and goal-scoring ability, earning multiple Ballon d'Or awards.",
        metadata={"topic": "Player", "player": "Lionel Messi", "source": "Sports Illustrated", "year": 2022}
    ),
    Document(
        page_content="The FIFA World Cup is held every four years and is the most prestigious international football tournament.",
        metadata={"topic": "Tournament", "name": "FIFA World Cup", "source": "FIFA", "year": 2022}
    ),
    Document(
        page_content="Tactics in football involve formations, pressing strategies, and player roles that determine how a team controls the game.",
        metadata={"topic": "Tactics", "source": "Coaching Manual", "year": 2021}
    ),
    Document(
        page_content="Cristiano Ronaldo is a legendary footballer celebrated for his athleticism, goal-scoring, and leadership on the field.",
        metadata={"topic": "Player", "player": "Cristiano Ronaldo", "source": "ESPN", "year": 2022}
    ),
    Document(
        page_content="Football clubs like FC Barcelona, Real Madrid, and Manchester United have millions of fans worldwide.",
        metadata={"topic": "Club", "source": "BBC Sport", "year": 2023}
    ),
    Document(
        page_content="The UEFA Champions League is an annual club competition that brings together the best European teams.",
        metadata={"topic": "Tournament", "name": "UEFA Champions League", "source": "UEFA", "year": 2023}
    ),
    Document(
        page_content="Youth development programs and football academies play a key role in nurturing the next generation of football stars.",
        metadata={"topic": "Development", "source": "The Guardian", "year": 2021}
    ),
]

## Step 2 — Embeddings & vector index

Create embeddings and a FAISS index. The index provides candidates to the base retriever which the compressor then shortens.

In [114]:
# Step 2: Initialize embedding model
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=gemini_api_key
)

In [115]:
# Step 3: Create Chroma vector store in memory
vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings
)

## Base retriever & compressor

Create a base retriever (top-k) and an LLM-based compressor (LLMChainExtractor). The `ContextualCompressionRetriever` will call the compressor to shorten each candidate before returning.

In [116]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [117]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=gemini_api_key
)

In [118]:
compressor = LLMChainExtractor.from_llm(model)

In [119]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

## Run query with compression

Invoke the `compression_retriever.invoke(query)` to get compressed passages. These are ideal for passing as context to a generation model.

In [120]:
query = "Tell me about the most influential players and tournaments in football."

In [121]:
compressed_results = compression_retriever.invoke(query)

In [122]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
The FIFA World Cup is held every four years and is the most prestigious international football tournament.

--- Result 2 ---
The UEFA Champions League is an annual club competition that brings together the best European teams.
